# HomeScraper

In [285]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

## Ouest France Immo

In [447]:
def ouest_france():
    
    def find_price(val):
        return val.find(class_="annPrix").text.strip().replace("\xa0", "").replace(" ", "").replace("€", "")

    def find_city(val):
        return val.find(class_="annVille").text

    def find_crit(val):
        return val.find(class_="annCriteres").text.strip()

    URL = 'https://www.ouestfrance-immo.com/acheter/maison/?lieux=12982,12984,13060,13176,12983,12959&vueMer=1'
    page = requests.get(URL)
    
    soup = BeautifulSoup(page.content, 'html.parser')
    results= soup.find(id="blocListAnnonces")
    bloc_res = results.find_all(class_='annLink')
    fix='https://www.ouestfrance-immo.com'
    
    all_val = []

    for val in results.find_all(class_='annLink', href=True):
        my_list = []

        my_list.append(find_price(val))
        my_list.append(find_city(val))
        crit = find_crit(val)
        surf=int(crit.split("|")[0].replace("m² ", ""))
        chb=int(crit.split("|")[1].replace("chb", ""))
        sdb=crit.split("|")[2].replace("sdb", "")

        my_list.append(surf)
        my_list.append(chb)
        my_list.append(sdb)

        my_list.append(val.find('img')['data-original'])
        my_list.append(fix+val['href'])

        page = requests.get(fix+val['href'])
        soup = BeautifulSoup(page.content, 'html.parser')
        results2 = soup.find_all('ul')

        terrain = soup.find_all('li')
        for val in terrain:

            if "Surf. terrain" in str(val):
                val = int(str(val).split("<strong>")[1].split("</strong>")[0].replace("m²", "").strip().replace(" ", ""))
                my_list.append(val)

        photos=results2[6]

        list_photos = []

        for pic in photos:
            if pic != '\n':
                list_photos.append(str(pic).split("src=")[1].split(" title=")[0])
        my_list.append(list_photos)
        all_val.append(my_list)
    
    df_of = pd.DataFrame(all_val)
    df_of.columns=["Prix", "Ville", "Surface", "Chambres", "Sdb", "Couverture", "Lien", "Terrain", "Photos"]
    df_of['VueMer'] = "Oui"
    df_of['Site'] = "OuestFrance"
    
    return df_of

In [451]:
df_of = ouest_france()
df_of

,Prix,Ville,Surface,Chambres,Sdb,Couverture,Lien,Terrain,Photos,VueMer,Site
0,366800,Lanvéoc,150,6,2,https://www.ouestfrance-immo.com/photo-vente-m...,https://www.ouestfrance-immo.com/immobilier/ve...,8544,"[""https://www.ouestfrance-immo.com/photo-vente...",Oui,OuestFrance
1,177500,Lanvéoc,330,4,1,https://www.ouestfrance-immo.com/photo-vente-m...,https://www.ouestfrance-immo.com/immobilier/ve...,546,"[""https://www.ouestfrance-immo.com/photo-vente...",Oui,OuestFrance
2,387600,Crozon,160,7,2,https://www.ouestfrance-immo.com/photo-vente-m...,https://www.ouestfrance-immo.com/immobilier/ve...,2022,"[""https://www.ouestfrance-immo.com/photo-vente...",Oui,OuestFrance
3,336000,Crozon,130,4,1,https://www.ouestfrance-immo.com/photo-vente-m...,https://www.ouestfrance-immo.com/immobilier/ve...,2000,"[""https://www.ouestfrance-immo.com/photo-vente...",Oui,OuestFrance
4,169600,Roscanvel,116,5,1,https://www.ouestfrance-immo.com/photo-vente-m...,https://www.ouestfrance-immo.com/immobilier/ve...,1904,"[""https://www.ouestfrance-immo.com/photo-vente...",Oui,OuestFrance
5,798000,morgat,190,6,2,https://www.ouestfrance-immo.com/photo-vente-m...,https://www.ouestfrance-immo.com/immobilier/ve...,1594,"[""https://www.ouestfrance-immo.com/photo-vente...",Oui,OuestFrance
6,279470,Crozon,82,2,1,https://www.ouestfrance-immo.com/photo-vente-m...,https://www.ouestfrance-immo.com/immobilier/ve...,1265,"[""https://www.ouestfrance-immo.com/photo-vente...",Oui,OuestFrance


## Agence de la Presqu'ile

In [449]:
def agence_presquile():

    URL = 'https://www.agencedelapresquile.com/achat-immobilier-crozon/type-1-maison/'
    page = requests.get(URL)

    soup = BeautifulSoup(page.content, 'html.parser')
    results= soup.find_all(class_="btn btn-default")

    fix = "https://www.agencedelapresquile.com"
    all_val = []

    for i in range(1, int(results[1].text)+1):

        URL = 'https://www.agencedelapresquile.com/achat-immobilier-crozon/type-1-maison/'+str(i)
        page = requests.get(URL)

        soup = BeautifulSoup(page.content, 'html.parser')

        results= soup.find_all(class_="block-link")
        link_thumbnail = list(soup.find_all(class_="mainImgLst3"))

        k = 0

        for link in results:

            page = requests.get(fix+link['href'])
            soup = BeautifulSoup(page.content, 'html.parser')
            results2 = soup.find_all(class_="data")

            photos = soup.find_all("img")
            link_photos = []
            for pic in photos:
                if ".jpg" in str(pic) and 'alt=""' in str(pic):
                    link_photos.append(pic['src'][2:])

            values = []

            prix = None
            ville = None
            surf = None
            chb = None
            terr = None

            for info in results2:

                if "Prix du bien" in str(info):
                    prix = int(info.find(class_="valueInfos").text.replace("€", "").strip().replace(" ", ""))
                elif "Ville" in str(info):
                    ville = info.find(class_="valueInfos").text.strip()
                elif "Surface habitable (m²)" in str(info):
                    surf = int(info.find(class_="valueInfos").text.replace("m²", "").strip().replace(" ", ""))
                elif "Nombre de chambre(s)" in str(info):
                    chb = info.find(class_="valueInfos").text.strip()
                elif "surface terrain" in str(info):
                    terr = int(info.find(class_="valueInfos").text.replace("m²", "").strip().replace(" ", ""))

            a_vue_mer = "Non"
            vue_mer = soup.find(itemprop="description").text
            if ("vue" in vue_mer or "bord" in vue_mer) and "mer" in vue_mer:
                a_vue_mer = "Oui"

            all_val.append([prix, ville, surf, chb, None, link_thumbnail[k]['src'][2:], fix+link['href'], terr, link_photos, a_vue_mer])
            k += 1
    df_pi = pd.DataFrame(all_val)
    df_pi.columns=["Prix", "Ville", "Surface", "Chambres", "Sdb", "Couverture", "Lien", "Terrain", "Photos", "VueMer"]
    df_pi['Site'] = "PresquileImmo"
    
    return df_pi

In [452]:
df_pi = agence_presquile()
df_pi

,Prix,Ville,Surface,Chambres,Sdb,Couverture,Lien,Terrain,Photos,VueMer,Site
0,361870,CROZON,104,3,None,ag-presquile.staticlbi.com/700x700/images/bien...,https://www.agencedelapresquile.com/318-propri...,2898.0,[ag-presquile.staticlbi.com/original/images/bi...,Oui,PresquileImmo
1,279470,CROZON,82,2,None,ag-presquile.staticlbi.com/700x700/images/bien...,https://www.agencedelapresquile.com/298-maison...,1265.0,[ag-presquile.staticlbi.com/original/images/bi...,Oui,PresquileImmo
2,567870,CROZON,214,4,None,ag-presquile.staticlbi.com/700x700/images/bien...,https://www.agencedelapresquile.com/297-villa-...,4684.0,[ag-presquile.staticlbi.com/original/images/bi...,Non,PresquileImmo
3,387600,CROZON,160,7,None,ag-presquile.staticlbi.com/700x700/images/bien...,https://www.agencedelapresquile.com/296-villa-...,2022.0,[ag-presquile.staticlbi.com/original/images/bi...,Non,PresquileImmo
4,433970,CROZON,130,5,None,ag-presquile.staticlbi.com/700x700/images/bien...,https://www.agencedelapresquile.com/291-ensemb...,3430.0,[ag-presquile.staticlbi.com/original/images/bi...,Non,PresquileImmo
5,320670,CROZON,110,3,None,ag-presquile.staticlbi.com/700x700/images/bien...,https://www.agencedelapresquile.com/289-maison...,1014.0,[ag-presquile.staticlbi.com/original/images/bi...,Non,PresquileImmo
6,207300,CROZON,155,3,None,ag-presquile.staticlbi.com/700x700/images/bien...,https://www.agencedelapresquile.com/285-maison...,570.0,[ag-presquile.staticlbi.com/original/images/bi...,Non,PresquileImmo
7,269170,CROZON,100,3,None,ag-presquile.staticlbi.com/700x700/images/bien...,https://www.agencedelapresquile.com/264-belle-...,793.0,[ag-presquile.staticlbi.com/original/images/bi...,Non,PresquileImmo
8,155870,CROZON,19,2,None,ag-presquile.staticlbi.com/700x700/images/bien...,https://www.agencedelapresquile.com/263-ancien...,701.0,[ag-presquile.staticlbi.com/original/images/bi...,Non,PresquileImmo
9,341270,CROZON,104,2,None,ag-presquile.staticlbi.com/700x700/images/bien...,https://www.agencedelapresquile.com/254-maison...,1678.0,[ag-presquile.staticlbi.com/original/images/bi...,Oui,PresquileImmo


# Concatenate

In [455]:
df_concat = pd.concat([df_of, df_pi])
df_concat

,Prix,Ville,Surface,Chambres,Sdb,Couverture,Lien,Terrain,Photos,VueMer,Site
0,366800,Lanvéoc,150,6,2,https://www.ouestfrance-immo.com/photo-vente-m...,https://www.ouestfrance-immo.com/immobilier/ve...,8544.0,"[""https://www.ouestfrance-immo.com/photo-vente...",Oui,OuestFrance
1,177500,Lanvéoc,330,4,1,https://www.ouestfrance-immo.com/photo-vente-m...,https://www.ouestfrance-immo.com/immobilier/ve...,546.0,"[""https://www.ouestfrance-immo.com/photo-vente...",Oui,OuestFrance
2,387600,Crozon,160,7,2,https://www.ouestfrance-immo.com/photo-vente-m...,https://www.ouestfrance-immo.com/immobilier/ve...,2022.0,"[""https://www.ouestfrance-immo.com/photo-vente...",Oui,OuestFrance
3,336000,Crozon,130,4,1,https://www.ouestfrance-immo.com/photo-vente-m...,https://www.ouestfrance-immo.com/immobilier/ve...,2000.0,"[""https://www.ouestfrance-immo.com/photo-vente...",Oui,OuestFrance
4,169600,Roscanvel,116,5,1,https://www.ouestfrance-immo.com/photo-vente-m...,https://www.ouestfrance-immo.com/immobilier/ve...,1904.0,"[""https://www.ouestfrance-immo.com/photo-vente...",Oui,OuestFrance
5,798000,morgat,190,6,2,https://www.ouestfrance-immo.com/photo-vente-m...,https://www.ouestfrance-immo.com/immobilier/ve...,1594.0,"[""https://www.ouestfrance-immo.com/photo-vente...",Oui,OuestFrance
6,279470,Crozon,82,2,1,https://www.ouestfrance-immo.com/photo-vente-m...,https://www.ouestfrance-immo.com/immobilier/ve...,1265.0,"[""https://www.ouestfrance-immo.com/photo-vente...",Oui,OuestFrance
0,361870,CROZON,104,3,None,ag-presquile.staticlbi.com/700x700/images/bien...,https://www.agencedelapresquile.com/318-propri...,2898.0,[ag-presquile.staticlbi.com/original/images/bi...,Oui,PresquileImmo
1,279470,CROZON,82,2,None,ag-presquile.staticlbi.com/700x700/images/bien...,https://www.agencedelapresquile.com/298-maison...,1265.0,[ag-presquile.staticlbi.com/original/images/bi...,Oui,PresquileImmo
2,567870,CROZON,214,4,None,ag-presquile.staticlbi.com/700x700/images/bien...,https://www.agencedelapresquile.com/297-villa-...,4684.0,[ag-presquile.staticlbi.com/original/images/bi...,Non,PresquileImmo


In [527]:
df_concat.columns

Index(['Prix', 'Ville', 'Surface', 'Chambres', 'Sdb', 'Couverture', 'Lien',
       'Terrain', 'Photos', 'VueMer', 'Site'],
      dtype='object')

## Cabinet Treguer

In [472]:
URL = 'http://www.cabinet-treguer.fr/advanced-search/page/1/?bedrooms&keyword=29160&max-area&max-price&min-area&status=vente&type=maison'
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')
results= soup.find_all("a")
results

[<a href="http://www.cabinet-treguer.fr/">
 <img alt="logo" src="http://s3.amazonaws.com/aivoni/wp-content/agence165589-images/20190916121923/logo-cabinet-treguer-min.png"/>
 </a>,
 <a href="http://www.cabinet-treguer.fr/">Accueil</a>,
 <a href="http://www.cabinet-treguer.fr/nos-agences-2/">Nos Agences</a>,
 <a href="http://www.cabinet-treguer.fr/vente/">Vente</a>,
 <a href="#">Location</a>,
 <a href="http://www.cabinet-treguer.fr/location-a-lannee/">Location à l’année</a>,
 <a href="http://www.cabinet-treguer.fr/location-de-vacances/">Location de vacances</a>,
 <a href="http://www.cabinet-treguer.fr/estimation-immobiliere/">Estimation</a>,
 <a href="http://www.cabinet-treguer.fr/notre-agence/">Notre équipe</a>,
 <a href="http://www.cabinet-treguer.fr/biens-vendus/">Biens vendus</a>,
 <a href="http://www.cabinet-treguer.fr/contact/">Contact</a>,
 <a href="/wp-admin"><i class="fa fa-user hidden-md hidden-lg"></i> <span class="hidden-sm hidden-xs">Connexion</span></a>,
 <a href="http://w

In [525]:
def treguet():
    
    URL = 'http://www.cabinet-treguer.fr/advanced-search/page/1/?bedrooms&keyword=29160&max-area&max-price&min-area&status=vente&type=maison'
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')

    num_pg = []
    for val in soup.find(class_="pagination-main").text:
        if val.isdigit():
            num_pg.append(int(val))

    num_pg = max(num_pg)
    res  = []
    
    for npage in range(1, num_pg+1):
        
        URL = 'http://www.cabinet-treguer.fr/advanced-search/page/%s/?bedrooms&keyword=29160&max-area&max-price&min-area&status=vente&type=maison'%str(npage)
        page = requests.get(URL)
        soup = BeautifulSoup(page.content, 'html.parser')
    
        results = soup.find_all(class_="btn btn-primary")
        results = results[::2]

        for val in results:


            link = val['href']
            page = requests.get(link)
            soup = BeautifulSoup(page.content, 'html.parser')

            place = soup.find(id="annonce-description")
            city = place.text.strip().split("–")[-1].split()[0].title()

            pic = soup.find_all(class_="sp-image")

            list_pics = []

            for p in pic:
                list_pics.append(p['src'])

            results2 = soup.find_all(class_="text-right")

            price = int(soup.find("strong").text.replace(" €", "").replace(".", ""))
            vue_mer = soup.find(id="descriptionblockxs").text

            to_ret = "Non"

            if ("vue" in vue_mer or "bord" in vue_mer) and "mer" in vue_mer:
                to_ret="Oui"

            feat = []
            for elem in results2[:4]:
                if str(elem.text) != "":
                    feat.append(int(elem.text.replace(" m²", "")))

            feat.append(price)
            feat.append(city)
            feat.append(link)
            feat.append(list_pics)
            feat.append(to_ret)

            res.append(feat)
    df=pd.DataFrame(res)
    df.columns=["Surface", "Terrain", "Chambres", "Prix", "Ville", "Lien", "Photos", "VueMer"]
    df['Site'] = "Treguer"
    df['Couverture'] = df['Photos'].apply(lambda x: x[0])
    
    df = df.reindex(['Prix', 'Ville', 'Surface', 'Chambres', 'Couverture', 'Lien',
       'Terrain', 'Photos', 'VueMer', 'Site'], axis=1)
        
    return df

In [528]:
df_treguet = treguet()
df_treguet

,Surface,Terrain,Chambres,Prix,Ville,Lien,Photos,VueMer,Site,Couverture
0,300,2022,4,399000,Crozon,http://www.cabinet-treguer.fr/property/maison-...,[http://s3.amazonaws.com/aivoni/wp-content/age...,Non,Treguer,http://s3.amazonaws.com/aivoni/wp-content/agen...
1,261,2006,8,798000,Crozon,http://www.cabinet-treguer.fr/property/maison-...,[http://s3.amazonaws.com/aivoni/wp-content/age...,Non,Treguer,http://s3.amazonaws.com/aivoni/wp-content/agen...
2,130,2000,4,336000,Crozon,http://www.cabinet-treguer.fr/property/maison-...,[http://s3.amazonaws.com/aivoni/wp-content/age...,Non,Treguer,http://s3.amazonaws.com/aivoni/wp-content/agen...
3,200,1400,6,1260000,Crozon,http://www.cabinet-treguer.fr/property/maison-...,[http://s3.amazonaws.com/aivoni/wp-content/age...,Oui,Treguer,http://s3.amazonaws.com/aivoni/wp-content/agen...
4,100,1203,2,197950,Lanveoc,http://www.cabinet-treguer.fr/property/maison-...,[http://s3.amazonaws.com/aivoni/wp-content/age...,Non,Treguer,http://s3.amazonaws.com/aivoni/wp-content/agen...
5,121,2212,5,420000,Crozon,http://www.cabinet-treguer.fr/property/maison-...,[http://s3.amazonaws.com/aivoni/wp-content/age...,Oui,Treguer,http://s3.amazonaws.com/aivoni/wp-content/agen...
6,148,2400,4,840000,Crozon,http://www.cabinet-treguer.fr/property/maison-...,[http://s3.amazonaws.com/aivoni/wp-content/age...,Non,Treguer,http://s3.amazonaws.com/aivoni/wp-content/agen...
7,97,884,4,129600,Crozon,http://www.cabinet-treguer.fr/property/maison-...,[http://s3.amazonaws.com/aivoni/wp-content/age...,Non,Treguer,http://s3.amazonaws.com/aivoni/wp-content/agen...
8,162,1182,4,181900,Crozon,http://www.cabinet-treguer.fr/property/maison-...,[http://s3.amazonaws.com/aivoni/wp-content/age...,Non,Treguer,http://s3.amazonaws.com/aivoni/wp-content/agen...
9,70,1026,2,135000,Crozon,http://www.cabinet-treguer.fr/property/maison-...,[http://s3.amazonaws.com/aivoni/wp-content/age...,Non,Treguer,http://s3.amazonaws.com/aivoni/wp-content/agen...


## Crozon Immobilier

In [573]:
URL = 'http://www.crozonimmobilier.com/catalog/advanced_search_result.php?action=update_search&search_id=1667485905895917&C_28_search=EGAL&C_28_type=UNIQUE&C_28=Vente&C_27_search=EGAL&C_27_type=TEXT&C_27=2&C_27_tmp=2&C_30_search=COMPRIS&C_30_type=NUMBER&C_30_MIN=&C_30_MAX=&C_65_search=CONTIENT&C_65_type=TEXT&C_65=&&search_id=1667485905895917&page=1'

page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')

In [608]:
a = soup.find_all(class_="link_img", href=True)
fix = 'http://www.crozonimmobilier.com/'

for val in a[:1]:

    link = fix + val['href'][3:]
    print(link)
    page = requests.get(link)
    soup2 = BeautifulSoup(page.content, 'html.parser')
    b = soup2.find_all(class_="col-sm-12")
    for val in b:
        v = val.text.strip()
        if v.startswith("Référence :"):
            list_char = val.text
            chb = list_char.split("Chambres : ")[1].split()[0]
            print(chb)
            surf = list_char.split("Surface habitable : ")[1].split()[0]
            print(surf)
            terr = list_char.split("Surface terrain : ")[1].split()[0]
            print(terr)    
    price = soup2.find(class_="hono_inclus_price").text
    
    import unidecode
    price = unidecode.unidecode(price)

    print(int(price.strip().replace("€", "").replace(" ", "")))

http://www.crozonimmobilier.com/fiches/26_36088909/lanveoc-cadre-agreable.html?search_id=1667485905895917&offset=0&page=1&
2
80
1250
199280


In [643]:
link = "https://www.bourse-immobilier.fr/achat-maison-crozon?quartiers=29042&surface=0&sterr=0&prix=-&typebien=1&nbpieces=1-2-3-4-5&og=0&where=Crozon-__29160_&_b=1&_p=0&tyloc=2&neuf=1&ancien=1&ids=29042"

page = requests.get(link)
soup = BeautifulSoup(page.content, 'html.parser')

fix = "https://www.bourse-immobilier.fr"

all_res = []

for l in soup.find_all(class_="bottom"):
    lk = l.find("a", href=True)['href']
    if "annonce" in lk:
        new_link = fix + lk
        
        page = requests.get(new_link)
        soup2 = BeautifulSoup(page.content, 'html.parser')
        
        surf = None
        terr = None
        chb = None
        price = None
        ville = None
        pic = None
        
        pr = soup2.find(id="detailprix").text
        price = int(pr.split(" : ")[1].split("-")[0].strip().replace("€", "").replace(" ", ""))
        city = soup2.find(class_="ville").text.split("|")[0].strip()
        
        pics = soup2.find_all(class_="img-responsive")
        pic_fix = "https://www.bourse-immobilier.fr/jpg/"
        list_p = []
        
        descr = soup2.find(id="visitez").text
        
        vue_mer = "Non"
        if ("vue" in descr or "bord" in descr or "accès" in descr) and ("mer" in descr or "grève" in descr or "baie" in descr):
            vue_mer = "Oui"
                
        for p in pics:
            list_p.append(p['data-lazy'])
        
        for char in soup2.find(id="caracteristique-bien").find_all("li"):
            
            if "Surface habitable" in str(char):
                surf = int(char.text.split(":")[1].strip().replace("m²", "").strip().replace(" ", ""))
            elif "Superficie terrain" in str(char):
                terr = int(char.text.split(":")[1].strip().replace("m²", "").strip().replace(" ", ""))
            elif "Chambre(s)" in str(char):
                chb = int(char.text.split(":")[1].strip().replace("m²", "").strip().replace(" ", ""))
                
        all_res.append([price, surf, terr, chb, new_link, city, list_p, vue_mer])
    
df = pd.DataFrame(all_res)

df.columns=["Prix", "Surface", "Terrain", "Chambres", "Lien", "Ville", "Photos", "VueMer"]
df['Site'] = "BourseImmo"
df['Couverture'] = df['Photos'].apply(lambda x: x[0])

df = df.reindex(['Prix', 'Ville', 'Surface', 'Chambres', 'Couverture', 'Lien',
   'Terrain', 'Photos', 'VueMer', 'Site'], axis=1)
df
    

In [645]:
df = pd.DataFrame(all_res)

df.columns=["Prix", "Surface", "Terrain", "Chambres", "Lien", "Ville", "Photos", "VueMer"]
df['Site'] = "BourseImmo"
df['Couverture'] = df['Photos'].apply(lambda x: x[0])

df = df.reindex(['Prix', 'Ville', 'Surface', 'Chambres', 'Couverture', 'Lien',
   'Terrain', 'Photos', 'VueMer', 'Site'], axis=1)
df

,Prix,Ville,Surface,Chambres,Couverture,Lien,Terrain,Photos,VueMer,Site
0,173340,Crozon,85,4,https://www.bourse-immobilier.fr/jpg/206-2646_...,https://www.bourse-immobilier.fr/annonce-achat...,760.0,[https://www.bourse-immobilier.fr/jpg/206-2646...,Non,BourseImmo
1,275600,Crozon,100,3,https://www.bourse-immobilier.fr/jpg/206-2638_...,https://www.bourse-immobilier.fr/annonce-achat...,793.0,[https://www.bourse-immobilier.fr/jpg/206-2638...,Oui,BourseImmo
2,270300,Crozon,147,5,https://www.bourse-immobilier.fr/jpg/206-2628_...,https://www.bourse-immobilier.fr/annonce-achat...,1253.0,[https://www.bourse-immobilier.fr/jpg/206-2628...,Non,BourseImmo
3,181050,Crozon,83,4,https://www.bourse-immobilier.fr/jpg/206-2627_...,https://www.bourse-immobilier.fr/annonce-achat...,963.0,[https://www.bourse-immobilier.fr/jpg/206-2627...,Non,BourseImmo
4,139750,Crozon,90,4,https://www.bourse-immobilier.fr/jpg/206-2624_...,https://www.bourse-immobilier.fr/annonce-achat...,770.0,[https://www.bourse-immobilier.fr/jpg/206-2624...,Non,BourseImmo
5,207675,Crozon,106,4,https://www.bourse-immobilier.fr/jpg/206-2623_...,https://www.bourse-immobilier.fr/annonce-achat...,1016.0,[https://www.bourse-immobilier.fr/jpg/206-2623...,Non,BourseImmo
6,296800,Crozon,120,4,https://www.bourse-immobilier.fr/jpg/206-2622_...,https://www.bourse-immobilier.fr/annonce-achat...,1200.0,[https://www.bourse-immobilier.fr/jpg/206-2622...,Non,BourseImmo
7,233200,Crozon,114,3,https://www.bourse-immobilier.fr/jpg/206-2603_...,https://www.bourse-immobilier.fr/annonce-achat...,900.0,[https://www.bourse-immobilier.fr/jpg/206-2603...,Non,BourseImmo
8,316500,Crozon,105,3,https://www.bourse-immobilier.fr/jpg/206-2574_...,https://www.bourse-immobilier.fr/annonce-achat...,637.0,[https://www.bourse-immobilier.fr/jpg/206-2574...,Non,BourseImmo
9,149800,Crozon,72,2,https://www.bourse-immobilier.fr/jpg/206-2561_...,https://www.bourse-immobilier.fr/annonce-achat...,120.0,[https://www.bourse-immobilier.fr/jpg/206-2561...,Non,BourseImmo
